In [ ]:
import subprocess
import os

In [4]:
# Set the path to your LAStools folder (adjust if not added to PATH)
las_tools_path = r"D:\LAStools\bin"  # Example on Windows

# Helper function to call a LAStools command
def run_tool(tool, args):
    tool_exe = os.path.join(las_tools_path, tool)
    command = [tool_exe] + args
    print("Running:", " ".join(command))
    subprocess.run(command, check=True)

In [ ]:


# 1. Classify ground points (DTM extraction)
run_tool("lasground.exe", [
    "-i", "data/Philadelphia_100.las",         # Input LAS/LAZ file
    "-o", "ground_classified.las"  # Output file with ground classified
])

# 2. Classify non-ground features (e.g. buildings and trees)
# This step may use lasclassify or other classification tools available in LAStools.
# (Make sure to adjust parameters based on your data and desired classification thresholds.)
run_tool("lasclassify.exe", [
    "-i", "ground_classified.las",
    "-o", "classified.las"
])

# 3. Extract DSM (using highest returns)
run_tool("las2dem.exe", [
    "-i", "classified.las",
    "-keep_class", "1 2 3 4 5 6",  # include all classes if needed
    "-step", "1",                 # resolution in meters (adjust as needed)
    "-o", "dsm.tif"
])

# 4. Extract DTM (using only ground points, typically classification 2)
run_tool("las2dem.exe", [
    "-i", "classified.las",
    "-keep_class", "2",
    "-step", "1",
    "-o", "dtm.tif"
])

# 5. Compute nDSM (normalized DSM)
# You can subtract dtm.tif from dsm.tif using GDAL tools or Python libraries (e.g., rasterio, numpy).
# For example, using GDAL command-line:
subprocess.run([
    "gdal_calc.py",
    "-A", "dsm.tif",
    "-B", "dtm.tif",
    "--outfile=nDSM.tif",
    "--calc=A-B",
    "--NoDataValue=0"
], check=True)

# 6. Extract building DSM (filtering for buildings, usually classification 6)
run_tool("las2dem.exe", [
    "-i", "classified.las",
    "-keep_class", "6",
    "-step", "1",
    "-o", "building_dsm.tif"
])

# 7. Extract tree DSM (filtering for vegetation, typically classification 5)
run_tool("las2dem.exe", [
    "-i", "classified.las",
    "-keep_class", "5",
    "-step", "1",
    "-o", "tree_dsm.tif"
])


Running: D:\LAStools\bin\lasground.exe -i data/Philadelphia_100.las -o ground_classified.las
